In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_excel("30-10-2024 CUSTOMER PRE ORDER & PRICE LIST.xlsx")
dataharga = pd.read_excel("Harga Jual v2.xlsx")
master = pd.read_excel("Catalogue Update.xlsx")

dataharga['Margin_Lusin'] = dataharga['Margin_Lusin'].str.replace('%','').str.replace(',','').astype(float) / 100
dataharga['Margin_Koli'] = dataharga['Margin_Koli'].str.replace('%','').str.replace(',','').astype(float) / 100
dataharga['Margin_Special'] = dataharga['Margin_Special'].str.replace('%','').str.replace(',','').astype(float) / 100

In [8]:
master.rename(columns={'Item No.':'ItemCode'}, inplace=True)
data = pd.merge(data, master[['ItemCode', 'Sub Item']], on='ItemCode', how='left').rename(columns={'Sub Item':'Sub_Item'})

In [9]:
data = data[['ItemCode', 'Sub_Item', 'Modal']]

In [10]:
fornewitem = dataharga.groupby('Sub_Item').mean().reset_index()
fornewitem.rename(columns={'Margin_Lusin':'Margin_Lusin_New', 'Margin_Koli':'Margin_Koli_New', 'Margin_Special':'Margin_Special_New'}, inplace=True)

C:\Users\SJK2525\AppData\Local\Temp\ipykernel_4568\1806041884.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  fornewitem = dataharga.groupby('Sub_Item').mean().reset_index()


In [11]:
final = pd.merge(data, dataharga, on='ItemCode', how='left').rename(columns={'Harga_Modal':'Modal Terakhir'})
final.fillna(0, inplace=True)


In [12]:
final = final[['Sub_Item_x','ItemCode', 'Modal', 'Modal Terakhir', 'Margin_Lusin', 'Margin_Koli', 'Margin_Special','Harga_Jual_Lusin', 'Harga_Jual_Koli', 'Harga_Jual_Special']].rename(columns={'Sub_Item_x':'Sub_Item'})

In [13]:
final[final['ItemCode'] == 'SSR-2302W2']

,Sub_Item,ItemCode,Modal,Modal Terakhir,Margin_Lusin,Margin_Koli,Margin_Special,Harga_Jual_Lusin,Harga_Jual_Koli,Harga_Jual_Special
18,SISIR GAGANG,SSR-2302W2,59034.170556,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
final = pd.merge(final, fornewitem[['Sub_Item', 'Margin_Lusin_New', 'Margin_Koli_New', 'Margin_Special_New']], on='Sub_Item', how='left')

In [15]:
final['HargaLusinFinal'] = np.where(final['Margin_Lusin_New'] == 0,(((final['Modal'] * final['Margin_Lusin']) + final['Modal'])/500).round() * 500 ,(((final['Modal'] * final['Margin_Lusin_New']) + final['Modal'])/500).round() * 500 )

final['HargaKoliFinal'] = np.where(final['Margin_Koli_New'] == 0,(((final['Modal'] * final['Margin_Koli'])+final['Modal'])/500).round() * 500 ,(((final['Modal'] * final['Margin_Koli_New'])+final['Modal'])/500).round() * 500 )

final['HargaSpecialFinal'] = np.where(final['Margin_Special_New'] == 0,(((final['Modal'] * final['Margin_Special'])+final['Modal'])/500).round() * 500 ,(((final['Modal'] * final['Margin_Special_New'])+final['Modal'])/500).round() * 500 )

In [18]:
final[final['ItemCode'] == 'SSR-2302W2']

,Sub_Item,ItemCode,Modal,Modal Terakhir,Margin_Lusin,Margin_Koli,Margin_Special,Harga_Jual_Lusin,Harga_Jual_Koli,Harga_Jual_Special,Margin_Lusin_New,Margin_Koli_New,Margin_Special_New,HargaLusinFinal,HargaKoliFinal,HargaSpecialFinal
18,SISIR GAGANG,SSR-2302W2,59034.170556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.366436,0.329378,0.297216,80500.0,78500.0,76500.0


In [ ]:
#Upper

itemcode = "iniadalahkecil"
itemcode = itemcode.upper()


final['ItemCode'] = final['ItemCode'].str.upper()

In [20]:
final.sample()

,Sub_Item,ItemCode,Modal,Modal Terakhir,Margin_Lusin,Margin_Koli,Margin_Special,Harga_Jual_Lusin,Harga_Jual_Koli,Harga_Jual_Special,Margin_Lusin_New,Margin_Koli_New,Margin_Special_New,HargaLusinFinal,HargaKoliFinal,HargaSpecialFinal
51,PEN MEKANIK,PEN-2793W,79696.130251,83780.49,0.2533,0.2294,0.2175,105000.0,103000.0,102000.0,0.351582,0.326222,0.305428,107500.0,105500.0,104000.0
